## Description

A tutorial to give an example for finding the criterion map. 

#### Package required for AMIRAL: 
- numpy
- matplotlib
- astropy
- maoppy --> but I need to think how to implement it because it is being set a bit differently
- decovbench --> 
- cython 

To implement the environment, import the environment from .yml file. (Check to see if it is the most-up-to-date version.)

In [ ]:
# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
#Change to your path
os.chdir("/Users/alau/Repo/amiral")
from amiral import instructment, utils, parameter, gradient, minimisation, array
from scipy.optimize import minimize 
%matplotlib inline

In [ ]:
# Global vairable
# Parameter to be modify

FLUX = 5e8         # Object total intensity [e-]
test_data_dir  = "/Users/alau/Data/amiral_fits/VESTA/"
image_name = "image_noise_20.fits"

To setup the PSF estimation, you first need to:
- a defintion of an ao system
- PSF parameter
- image you would like to perform estimation

In [ ]:
aosys_dict = {
    'diameter': 7 , 
    'occ_ratio': 0.1 , 
    'no_acutuator' : 30, 
    'wavelength': 500, 
    'dimension': 256,
    'resolution_rad' : 3.5e-8 
}

In [ ]:
amiral_dict = {
    "r0": 0.15,  #0.2                
    "background": 0.01,      
    "amplitude": 1.2,  #1.6     
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

As you can see in here, PSF hyperparameters are not initialised. Therefore, it will be dealt with later on. Now convert the dict into arrays

In [ ]:
psf_keys, psf_guess = utils.dict2array(amiral_dict)

In [ ]:
img = utils.load_fits(test_data_dir+image_name)

In [ ]:
aosys_cls = instructment.aoSystem( 
        diameter = aosys_dict['diameter'], 
        occ_ratio = aosys_dict['occ_ratio'], 
        no_acutuator= aosys_dict['no_acutuator'], 
        wavelength = aosys_dict['wavelength']*1e-9, 
        resolution_rad = aosys_dict['resolution_rad'], 
        dimension=aosys_dict['dimension'])  


print(aosys_cls.N_padded)


img_trim = img[256-128:256+128,256-128:256+128]

# amiralparam = parameter.amiralParam(img ,guess = psf_guess, aosys = aosys_cls)
amiralparam = parameter.amiralParam(img_trim ,guess = psf_guess, aosys = aosys_cls)
plt.imshow(img_trim)

Now, we need to set up the bounds, hyperparameters and the fourier variables before calculating the criterion. 

In [ ]:
hyper_guess = amiralparam.hyperparam_initial(psf_guess)
hyper_min, hyper_max = amiralparam.hyperparam_bound(psf_guess, p_upperbound = 4.5)

psf_guess[-3] = hyper_guess[0] 
psf_guess[-2] = hyper_guess[1]
psf_guess[-1] = hyper_guess[2]

param_min = np.asarray([0.01,0,0,1e-8,1.01])
param_max =  np.asarray([1.,1e8,1e8,1e3,10])

upperbound = np.concatenate((param_max, hyper_max))
lowerbound = np.concatenate((param_min, hyper_min))

param_numerical_condition = np.array([1., 1e-4, 1., 1., 1.])
hyperparam_numerical_condition = np.array([hyper_guess[0], hyper_guess[1], 1.])

numerical_condition = np.concatenate((param_numerical_condition, hyperparam_numerical_condition))

param_mask = np.asarray([1,0,1,0,0])
hyper_param_mask = np.asarray([1,1,0])

mask = np.concatenate((param_mask,hyper_param_mask))

amiral_cls = parameter.amiral(img=img_trim, guess=psf_guess, aosys = aosys_cls, upperbound = upperbound, lowerbound= lowerbound, numerical_condition = numerical_condition, fourier_variable = amiralparam.fourier_variable, mask = mask)


In [ ]:
numeric_param = minimisation.param_physical2numerical(psf_guess, amiral_cls.numerical_condition)

In [ ]:
print(psf_guess)
print(amiral_cls.numerical_condition)

In [ ]:
crit = amiral_cls.marg_criterion(numeric_param)
grad = amiral_cls.gradient(numeric_param, debug = True)

In [ ]:
crit = amiral_cls.marg_criterion(numeric_param)
print(crit-3002362.4)

In [ ]:
est_criterion, value_criterion, value_grad = amiral_cls.minimisation(psf_guess)


In [ ]:
print(est_criterion)

In [ ]:
est_criterion_1, value_criterion, value_grad = amiral_cls.minimisation(est_criterion)

In [ ]:
print(est_criterion_1)

In [ ]:
psf_guess

In [ ]:
grad = amiral_cls.gradient(psf_guess, debug = True)

In [ ]:
grad_norm = amiral_cls.gradient(psf_guess)